## A Portfolio of Technology Stocks in the S&P 500 using Markowitz's Modern Portfolio Theory

The objective of the project is look for the optimal portfolio which give best risk-return trade-off that lie along the minimum variance frontier within a set of given risky assets.

The assets in this case are the Technology Stocks of the S&P 500

#### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from scipy import optimize
import warnings
warnings.filterwarnings('ignore')

<br>

**The Historical stock datasets was downloaded in the form of cvs files from Yahoo Finanace and in the cell below, the data was read inton a Pandas dataframe**

**A much faster approach is an API call, one of which is the use of a library such as yfinance to get the stock data from Yahoo finance**

In [75]:
# Get a list of all CSV files in a directory
csv_files = glob.glob('data/*.csv')

# Create an empty dataframe to store the combined data

combined_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    file_name = str(csv_file.split('.')[0].split('\\')[-1])
    updated_df = df[['Date', 'Close']]
    updated_df[file_name] = updated_df['Close']
    updated_df.drop(['Close'], axis=1, inplace=True)
    updated_df['Date'] = pd.to_datetime(arg=updated_df['Date'])
    updated_df.set_index('Date', inplace=True)
    
    
    combined_df = pd.concat([combined_df, updated_df], axis=1)

    
# While downloading the dataset, the ending dates considered for the stock was 30th of August but some
# stocks had their end dates beyond that.The line of code was used to make the stock end dates uniform
combined_df.drop([combined_df.index[-1], combined_df.index[-2]], inplace=True)


# Reversing the order of the sorting

<br>
<br>

**Viewing the dataset**

In [3]:
combined_df

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANET,ANSS,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,14.686786,52.950001,28.570000,36.029999,30.809999,32.930000,10.700000,5.480000,NaN,57.389999,...,16.820000,55.820000,31.480000,13.750000,20.815001,29.760000,30.340000,35.820000,30.980000,35.720001
2012-01-04,14.765714,52.930000,28.280001,35.980000,30.209999,32.439999,10.760000,5.460000,NaN,56.400002,...,16.780001,55.790001,31.570000,14.180000,20.105000,29.570000,30.360001,35.820000,31.299999,35.450001
2012-01-05,14.929643,51.919998,28.480000,36.139999,30.820000,32.430000,10.830000,5.460000,NaN,57.299999,...,17.280001,55.570000,31.920000,14.620000,20.385000,29.780001,31.090000,35.740002,32.759998,35.400002
2012-01-06,15.085714,51.830002,28.719999,35.900002,30.959999,32.759998,11.010000,5.430000,NaN,56.939999,...,17.150000,55.580002,32.360001,14.630000,20.075001,29.780001,31.809999,35.400002,33.490002,35.110001
2012-01-09,15.061786,51.740002,28.530001,36.599998,31.080000,32.610001,11.340000,5.590000,NaN,56.680000,...,17.809999,55.220001,32.810001,14.890000,20.870001,30.160000,31.559999,35.020000,33.750000,34.950001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,176.380005,314.140015,512.429993,173.690002,208.600006,101.650002,142.520004,101.800003,179.190002,298.559998,...,104.230003,406.000000,127.559998,102.110001,52.259998,165.589996,375.119995,202.570007,39.869999,265.920013
2023-08-25,178.610001,318.760010,525.059998,178.160004,215.919998,102.820000,144.360001,102.250000,180.899994,304.890015,...,105.389999,408.549988,128.639999,103.779999,52.150002,167.809998,378.790009,205.740005,39.490002,268.390015
2023-08-28,180.190002,320.910004,529.919983,180.330002,217.089996,103.220001,145.419998,102.610001,181.960007,308.149994,...,106.199997,416.130005,129.750000,104.570000,53.259998,168.699997,384.250000,206.529999,41.410000,269.920013


<br>
<br>
<br>

The historical stock data considered spanned ten years, using daily data - from `01-01-2012` to `30-08-2023`

The start date for stock was `03-01-2012` as the first two days of 2012 were public holidays.

According to Google, there are approximately `252 days in a year`, `21 days per month` and `12 months per year` for trading in a Stock market.

Some Technology companies on the S&P 500 do not have their stock options extend back to 2012, and they were therfore dropped from the dataset used leaving us with 52 portfolios

In [81]:
# Dropping portfolios that are not up to 10 years

combined_df.dropna(axis=1, how='any', inplace=True)

In [82]:
combined_df.sort_values(['Date'], ascending=False)

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-30,187.649994,323.250000,545.359985,181.570007,221.550003,104.639999,150.949997,106.589996,315.959991,892.280029,...,107.980003,420.070007,131.929993,107.370003,54.250000,169.229996,397.959991,209.119995,42.520000,279.109985
2023-08-29,184.119995,323.450012,540.570007,182.559998,219.399994,104.400002,149.979996,105.919998,314.209991,889.929993,...,107.779999,417.910004,131.190002,107.370003,54.240002,170.690002,391.589996,207.350006,41.549999,273.489990
2023-08-28,180.190002,320.910004,529.919983,180.330002,217.089996,103.220001,145.419998,102.610001,308.149994,861.080017,...,106.199997,416.130005,129.750000,104.570000,53.259998,168.699997,384.250000,206.529999,41.410000,269.920013
2023-08-25,178.610001,318.760010,525.059998,178.160004,215.919998,102.820000,144.360001,102.250000,304.890015,851.820007,...,105.389999,408.549988,128.639999,103.779999,52.150002,167.809998,378.790009,205.740005,39.490002,268.390015
2023-08-24,176.380005,314.140015,512.429993,173.690002,208.600006,101.650002,142.520004,101.800003,298.559998,854.460022,...,104.230003,406.000000,127.559998,102.110001,52.259998,165.589996,375.119995,202.570007,39.869999,265.920013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-01-09,15.061786,51.740002,28.530001,36.599998,31.080000,32.610001,11.340000,5.590000,56.680000,30.219999,...,17.809999,55.220001,32.810001,14.890000,20.870001,30.160000,31.559999,35.020000,33.750000,34.950001
2012-01-06,15.085714,51.830002,28.719999,35.900002,30.959999,32.759998,11.010000,5.430000,56.939999,28.309999,...,17.150000,55.580002,32.360001,14.630000,20.075001,29.780001,31.809999,35.400002,33.490002,35.110001
2012-01-05,14.929643,51.919998,28.480000,36.139999,30.820000,32.430000,10.830000,5.460000,57.299999,28.490000,...,17.280001,55.570000,31.920000,14.620000,20.385000,29.780001,31.090000,35.740002,32.759998,35.400002


In [83]:
Tickers = combined_df.columns

In [84]:
len(Tickers)

52

<br>
<br>

# Exploratory Data Analysis

**Viewing the info of the dataset to ensure that the available data is consistent**

In [6]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2934 entries, 2012-01-03 to 2023-08-30
Data columns (total 52 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2934 non-null   float64
 1   ACN     2934 non-null   float64
 2   ADBE    2934 non-null   float64
 3   ADI     2934 non-null   float64
 4   ADSK    2934 non-null   float64
 5   AKAM    2934 non-null   float64
 6   AMAT    2934 non-null   float64
 7   AMD     2934 non-null   float64
 8   ANSS    2934 non-null   float64
 9   AVGO    2934 non-null   float64
 10  CDNS    2934 non-null   float64
 11  CRM     2934 non-null   float64
 12  CSCO    2934 non-null   float64
 13  CTSH    2934 non-null   float64
 14  FFIV    2934 non-null   float64
 15  FICO    2934 non-null   float64
 16  FSLR    2934 non-null   float64
 17  FTNT    2934 non-null   float64
 18  GEN     2934 non-null   float64
 19  GLW     2934 non-null   float64
 20  HPQ     2934 non-null   float64
 21  IBM     2934 non-nu

# Implementation of the Markowitz's Modern Portfolio Theory

52 Portfolios of Technology companies from the S&P 500 are being considered

#### Important parameters


`Expected return for a portfolio` = `Transpose of weights of the assets within the portfolio` * `Expected return`

`StandardDeviation` or `Volatility` = squar root of (`Transpose of Weights of assets` * `variance-covariance` * `Weights of assests`)

`Risk free rate` is a return an investor can expect from an investment, given the risk associated is zero.

`Sharpe ratio` is a measure of risk-adjusted return

`Sharpe ratio` = (`ExpectedReturn` - `RiskFreeRate`) / `Variance`



<br>

### Daily Returns

<br>

The returns using the daily dataset was obtained for the individual portfolio.

In [7]:
RiskFreeRate = 3.8/100.0

In [8]:
# Get a list of all CSV files in a directory
csv_files = glob.glob('data/*.csv')

# Create an empty dataframe to store the combined data

daily_returns = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    file_name = str(csv_file.split('.')[0].split('\\')[-1])
    if file_name in list(combined_df.columns):
        ind_daily_return = pd.DataFrame(combined_df[file_name].pct_change())
        daily_returns = pd.concat([daily_returns, ind_daily_return], axis=1)

In [9]:
# viewing the origin dataframe after calculating daily returns

daily_returns

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,0.005374,-0.000378,-0.010150,-0.001388,-0.019474,-0.014880,0.005607,-0.003650,-0.017250,-0.017271,...,-0.002378,-0.000537,0.002859,0.031273,-0.034110,-0.006384,0.000659,0.000000,0.010329,-0.007559
2012-01-05,0.011102,-0.019082,0.007072,0.004447,0.020192,-0.000308,0.006506,0.000000,0.015957,0.001406,...,0.029797,-0.003943,0.011086,0.031030,0.013927,0.007102,0.024045,-0.002233,0.046645,-0.001410
2012-01-06,0.010454,-0.001733,0.008427,-0.006641,0.004542,0.010176,0.016620,-0.005495,-0.006283,-0.006318,...,-0.007523,0.000180,0.013784,0.000684,-0.015207,0.000000,0.023159,-0.009513,0.022283,-0.008192
2012-01-09,-0.001586,-0.001736,-0.006616,0.019498,0.003876,-0.004579,0.029973,0.029466,-0.004566,0.067467,...,0.038484,-0.006477,0.013906,0.017772,0.039601,0.012760,-0.007859,-0.010735,0.007763,-0.004557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,-0.026170,-0.003805,-0.034444,-0.021354,0.020748,-0.007712,-0.037027,-0.069725,-0.014100,-0.025234,...,-0.027977,-0.013941,-0.010933,-0.030479,-0.018038,-0.024966,-0.013673,-0.012769,-0.028745,-0.015439
2023-08-25,0.012643,0.014707,0.024647,0.025736,0.035091,0.011510,0.012910,0.004420,0.021202,-0.003090,...,0.011129,0.006281,0.008467,0.016355,-0.002105,0.013407,0.009784,0.015649,-0.009531,0.009289
2023-08-28,0.008846,0.006745,0.009256,0.012180,0.005419,0.003890,0.007343,0.003521,0.010692,0.010871,...,0.007686,0.018553,0.008629,0.007612,0.021285,0.005304,0.014414,0.003840,0.048620,0.005701


In [10]:
# This was written to view the dataframe from the current to the earliest dates, that is, reversing the order of the dataframe
# And the multiplication by 100 was done to express the returns as percentages which is the required way to express them

daily_returns = daily_returns.sort_values(['Date'], ascending=False)
daily_returns.head().mul(100)

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-30,0.019172,-0.000618,0.008861,-0.005423,0.009799,0.002299,0.006468,0.006326,0.005570,0.002641,...,0.001856,0.005169,0.005641,0.000000,0.000184,-0.008554,0.016267,0.008536,0.023345,0.020549
2023-08-29,0.021810,0.007915,0.020097,0.012366,0.010641,0.011432,0.031357,0.032258,0.019666,0.033504,...,0.014878,0.004278,0.011098,0.026776,0.018400,0.011796,0.019102,0.003970,0.003381,0.013226
2023-08-28,0.008846,0.006745,0.009256,0.012180,0.005419,0.003890,0.007343,0.003521,0.010692,0.010871,...,0.007686,0.018553,0.008629,0.007612,0.021285,0.005304,0.014414,0.003840,0.048620,0.005701
2023-08-25,0.012643,0.014707,0.024647,0.025736,0.035091,0.011510,0.012910,0.004420,0.021202,-0.003090,...,0.011129,0.006281,0.008467,0.016355,-0.002105,0.013407,0.009784,0.015649,-0.009531,0.009289
2023-08-24,-0.026170,-0.003805,-0.034444,-0.021354,0.020748,-0.007712,-0.037027,-0.069725,-0.014100,-0.025234,...,-0.027977,-0.013941,-0.010933,-0.030479,-0.018038,-0.024966,-0.013673,-0.012769,-0.028745,-0.015439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-01-09,-0.001586,-0.001736,-0.006616,0.019498,0.003876,-0.004579,0.029973,0.029466,-0.004566,0.067467,...,0.038484,-0.006477,0.013906,0.017772,0.039601,0.012760,-0.007859,-0.010735,0.007763,-0.004557
2012-01-06,0.010454,-0.001733,0.008427,-0.006641,0.004542,0.010176,0.016620,-0.005495,-0.006283,-0.006318,...,-0.007523,0.000180,0.013784,0.000684,-0.015207,0.000000,0.023159,-0.009513,0.022283,-0.008192
2012-01-05,0.011102,-0.019082,0.007072,0.004447,0.020192,-0.000308,0.006506,0.000000,0.015957,0.001406,...,0.029797,-0.003943,0.011086,0.031030,0.013927,0.007102,0.024045,-0.002233,0.046645,-0.001410


<br>
<br>

### Monthly returns for individual asset

<br>

The percentage returns per month for each portfolio.

**The `'M'` passed into the resample function was used to achieve this**

In [11]:
# Get a list of all CSV files in a directory
csv_files = glob.glob('data/*.csv')

# Create an empty dataframe to store the combined data

monthly_returns = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    file_name = str(csv_file.split('.')[0].split('\\')[-1])
    if file_name in list(combined_df.columns):
        ind_monthly_return = pd.DataFrame(combined_df[file_name].resample('M').ffill().pct_change())
        monthly_returns = pd.concat([monthly_returns, ind_monthly_return], axis=1)


In [109]:
monthly_returns = monthly_returns.sort_values(['Date'], ascending=False)
monthly_returns.head().mul(100)

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-31,-4.479513,2.181127,-0.148305,-9.001149,4.509646,10.730158,-0.422191,-6.826928,-7.641045,-0.708840,...,-5.587129,9.242454,-8.056319,-4.931821,0.836433,-5.983336,0.335322,-0.867505,-0.093987,-9.368102
2023-07-31,1.278546,2.517992,11.693489,2.422874,3.606864,5.151883,4.877545,0.430162,3.581923,3.599141,...,3.324601,-6.465420,2.375857,1.446151,1.624481,-0.011112,-4.763848,-6.646902,12.206699,4.100329
2023-06-30,9.433005,0.869500,17.042050,9.634755,2.617986,-2.442466,8.432100,-3.637590,2.064339,7.360509,...,6.936528,5.778976,14.435010,11.118874,13.434757,3.531170,4.914856,1.186635,-2.065582,12.667090
2023-05-31,4.461343,9.144102,10.655263,-1.217480,2.361521,12.382581,17.933296,32.270331,3.080397,28.964090,...,-2.256846,-6.213803,0.089891,9.641065,-0.912951,3.995220,4.730494,0.685304,12.456449,-8.839354
2023-04-30,2.898726,-1.931349,-2.026623,-8.792210,-6.422949,4.687098,-7.978509,-8.815426,-5.673070,-2.344356,...,-10.239024,-7.367666,-6.694618,-15.003260,-10.148799,-10.112359,6.877392,4.954337,-8.574460,-9.424529


<br>

### Average

In [13]:
# Average daily returns. Should be multiplied by 21 if used for monthly calculation
# and by 252 when used for annual calculations
AverageDailyReturn = pd.DataFrame(daily_returns.mean(), columns=['AverageDailyReturn'])


#Should be multiplied by 12 when used for annual calculations
AverageMonthlyReturn = pd.DataFrame(monthly_returns.mean(), columns=['AverageMonthlyReturn'])


# Average annual return computed from the AverageMonthlyReturn
AverageAnnualReturn = AverageMonthlyReturn.mul(12)

### Variance

In [14]:
# Variance of daily returns. Should be multiplied by 21 if used for monthly calculations
# and by 252 when used for annual calculations
DailyVariance = pd.DataFrame(daily_returns.var(), columns=['DailyReturnVariance'])


#Should be multiplied by 12 when used for annual calculations
MonthlyVariance = pd.DataFrame(monthly_returns.var(), columns=['MonthlyReturnVariance'])


# Variance of annual return computed from the AnnualMonthlyAverage
AnnualVariance = MonthlyVariance.mul(12)

In [15]:
# A Dataframe to visualize the averages and variances
AverageAndVariance = pd.concat([AverageDailyReturn, AverageMonthlyReturn,
                                  DailyVariance, MonthlyVariance], axis=1)

AverageAndVariance['AverageAnnualReturn'] = AverageAnnualReturn


AverageAndVariance['AnnualVariance'] = AnnualVariance

In [16]:
AverageAndVariance

,AverageDailyReturn,AverageMonthlyReturn,DailyReturnVariance,MonthlyReturnVariance,AverageAnnualReturn,AnnualVariance
AAPL,0.001033,0.020930,0.000328,0.006497,0.251157,0.077958
ACN,0.000730,0.014415,0.000226,0.003803,0.172976,0.045634
ADBE,0.001197,0.023715,0.000382,0.005720,0.284575,0.068642
ADI,0.000719,0.013579,0.000335,0.005074,0.162945,0.060883
ADSK,0.000930,0.017751,0.000513,0.009366,0.213013,0.112397
AKAM,0.000610,0.012109,0.000434,0.007390,0.145308,0.088682
AMAT,0.001175,0.022748,0.000542,0.009339,0.272975,0.112071
AMD,0.001659,0.033303,0.001315,0.027146,0.399639,0.325754
ANSS,0.000743,0.014384,0.000323,0.004896,0.172603,0.058757
AVGO,0.001405,0.027136,0.000470,0.006930,0.325626,0.083164


<br>

### Returns

The average annual return will be used to as the returns for subsequent calculations

In [17]:
Returns = AverageAnnualReturn

Returns

,AverageMonthlyReturn
AAPL,0.251157
ACN,0.172976
ADBE,0.284575
ADI,0.162945
ADSK,0.213013
AKAM,0.145308
AMAT,0.272975
AMD,0.399639
ANSS,0.172603
AVGO,0.325626


## Variance-Covariance Matrix

In [18]:
# Using the monthly returns, we compute the covariance matrix

covariance = monthly_returns.cov().mul(12)
covariance

,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANSS,AVGO,...,SWKS,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDC,ZBRA
AAPL,0.077958,0.032413,0.038195,0.029397,0.042452,0.025114,0.031819,0.056399,0.036636,0.032825,...,0.054344,0.021839,0.033646,0.038308,0.045134,0.027593,0.019414,0.025341,0.035208,0.049329
ACN,0.032413,0.045634,0.032013,0.026713,0.043775,0.022340,0.038707,0.049962,0.034070,0.029284,...,0.025112,0.024507,0.028197,0.036819,0.041107,0.025448,0.024971,0.026655,0.036663,0.040212
ADBE,0.038195,0.032013,0.068642,0.032274,0.053223,0.029140,0.044273,0.064532,0.037573,0.033175,...,0.036227,0.024207,0.035972,0.046826,0.049550,0.030900,0.029974,0.027193,0.042193,0.047603
ADI,0.029397,0.026713,0.032274,0.060883,0.044818,0.010936,0.058011,0.053830,0.033535,0.038968,...,0.041410,0.025008,0.039418,0.054322,0.040384,0.041683,0.017071,0.020744,0.053058,0.051326
ADSK,0.042452,0.043775,0.053223,0.044818,0.112397,0.021766,0.060765,0.069380,0.045987,0.043037,...,0.042669,0.035195,0.047651,0.055215,0.072882,0.038979,0.037770,0.041812,0.056221,0.066872
AKAM,0.025114,0.022340,0.029140,0.010936,0.021766,0.088682,0.027237,0.027781,0.015471,0.020446,...,0.024073,0.019463,0.019633,0.031297,0.027940,0.018936,0.008259,0.022920,0.042614,0.023162
AMAT,0.031819,0.038707,0.044273,0.058011,0.060765,0.027237,0.112071,0.090415,0.041522,0.054017,...,0.053358,0.037767,0.055170,0.079321,0.065057,0.050705,0.026727,0.028002,0.076651,0.068375
AMD,0.056399,0.049962,0.064532,0.053830,0.069380,0.027781,0.090415,0.325754,0.063969,0.065455,...,0.041167,0.040827,0.057631,0.069468,0.088928,0.049732,0.039475,0.046444,0.061806,0.072709
ANSS,0.036636,0.034070,0.037573,0.033535,0.045987,0.015471,0.041522,0.063969,0.058757,0.029785,...,0.034431,0.026863,0.034291,0.038965,0.041272,0.028274,0.030976,0.028635,0.033982,0.046867
AVGO,0.032825,0.029284,0.033175,0.038968,0.043037,0.020446,0.054017,0.065455,0.029785,0.083164,...,0.055161,0.019015,0.035521,0.050573,0.032456,0.030173,0.016247,0.021837,0.049216,0.043887


## Equally-Weighted Portfolio

We start with equally-weighted portfolio and observe obtained results


The sum of the individual weights is expected to be equal to 1

In [19]:
# This function creates a equal weights based on the size of the portfolio.

def equal_weights(df):
    data = []
    weight = 1/len(list(df.columns))
    for col in df.columns:
        data.append([col, weight])
    
    weight_df = pd.DataFrame(data, columns=['Ticker', 'Weights'])
    
    
    return weight_df.set_index('Ticker')

In [20]:
weight_df = equal_weights(combined_df)
weight_df.head()

,Weights
Ticker,
AAPL,0.019231
ACN,0.019231
ADBE,0.019231
ADI,0.019231
ADSK,0.019231


<br>

### Expected Returns

**This is the amount of profit or loss an investor can anticipate receiving on an investment.**

It is the product of Weights and returns

In [21]:
ExpectedReturn = weight_df.transpose().dot(Returns)

ExpectedReturn.values[0][0]

0.20337137246796205

#### Expected Return in percentage

In [22]:
ExpectedReturnPercentage = ExpectedReturn['AverageMonthlyReturn'].apply('{:.2%}'.format)[0]

print(ExpectedReturnPercentage)

20.34%


<br>

### Standard Deviation

**This is the statistical measure of market volatility, measuring how widely prices are dispersed from the average price.**

In [74]:
weight_df.shape

(52, 1)

In [72]:
StandardDeviation = weight_df.transpose().dot(covariance).dot(weight_df).apply(np.sqrt,axis=1)

StandardDeviation.values[0][0]

0.193843601195992

#### Standard Deviation expressed as a percentage

In [24]:
StandardDeviationPercentage = StandardDeviation['Weights'].apply('{:.2%}'.format)[0]

print(StandardDeviationPercentage)

19.38%


<br>

### Sharpe Ratio

**It is the measure of risk-adjusted return of a financial portfolio.**

**A portfolio with a higher Sharpe ratio is considered superior relative to its peers**

**The `Risk Free Rate` for the US for the month of August is estimated to be `3.8%`**

In [25]:
SharpeRatio = (ExpectedReturn.values[0][0] - RiskFreeRate)/StandardDeviation.values[0][0]

SharpeRatio

0.8531175207623068

<br>
<br>

## Maximizing the Sharpe Ratio to obtain Optimal Portfolio Weight

To obtain an optimally-risky portfolio, it is essential to maximize the `Sharpe Ratio`

The `minimize` library function from the `Scipy.optimize` package of Python was used.

#### Objective:
 - The objective is to find out the optimal weights that will ensure maximal Sharpe Ratio for the portfolio.
 - This is required to explore the optimal portfolio set.

Since Scipy Optimization Library only minimize, and the objective is to maximize, the **principle of duality** from Optimization Theory can be used to undertake the transformation to obtain maximization.

Reference: https://www.kaggle.com/code/vijipai/lesson-6-sharpe-ratio-based-portfolio-optimization

#### The Function below was written to maximize Sharpe Ratio

In [37]:
# Function to undertake Sharpe Ratio maximization subject to basic constraints of the portfolio

def MaximizeSharpeRatio(AverageReturns, VarianceCovariancce, RiskFreeRate, PortfolioSize):
    '''
            A function to perform the optimization of a given function
    Inputs:
            AverageReturns: A one-dimensional array of sihape n x 1; in this case annual average of portfolio
            VarianceCovariance: A two-dimensional array of shape n x n; covariance matrix of portfolio
            RiskFreeRate: A scalar value (float)
            PortfolioSize: An intger; The of the portfolio
            
    Outputs:
            OptimizedSharpeRatio: A float representing the optimized function; in this instance, Sharpe Ratio
            
    '''
    
    # define maximization of Sharpe Ratio using principle of duality
    
    def  ObjectiveFunction(Weights, AverageReturns, VarianceCovariancce, RiskFreeRate, PortfolioSize):
        '''
                This is the function to be maximixed passed into the minimize funtion of the Scipy.optimize
        Inputs:
                Weights: A one-dimensional array of shape n x 1 that is to be adjusted for optimization
                
                Other inputs are consistent with the above description
        
        Output:
                func: The function to be optimized (Maximised).
        '''
        
        ExpectedReturn = Weights.transpose().dot(AverageReturns)
        StandardDeviation = np.sqrt(Weights.transpose().dot(VarianceCovariancce).dot(Weights))
        funcDenom = StandardDeviation
        funcNumer = ExpectedReturn - RiskFreeRate
        
        func = -(funcNumer / funcDenom)
        return func

    
    # define equality constraint representing fully invested portfolio
    def constraintEq(Weights):
        
        A = np.ones(Weights.shape)
        b = 1
        constraintVal = np.matmul(A, Weights.T)-b 
        return constraintVal
    
      
    
    #define bounds and other parameters
    InitialWeights = np.repeat(1/PortfolioSize, PortfolioSize)
    cons = ({'type': 'ineq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb, ub) for x in InitialWeights])
    
    #invoke minimize solver
    OptimizedSharpeRatio = optimize.minimize(ObjectiveFunction, x0 = InitialWeights, args = (AverageReturns, VarianceCovariancce, 
                                                    RiskFreeRate, PortfolioSize), method = 'SLSQP', 
                                                     bounds = bnds, constraints = cons, tol = 10**-3)
    
    return OptimizedSharpeRatio

In [108]:
PortfolioSize = len(Returns)

<br>
<br>

#### Maximized Sharpe Ratio and optimal weights computation

In [186]:
# Compute maximal Sharpe Ratio and optimal weights

def RiskAndSharpeRatio(MaximizedSharpeRatioResult, covariance, RiskFreeRate):
    '''
        This function computes the risk returns and max Sharpe Ratio of the optimal portfolio
        
        Input:
                MaximizedSharpeRatioResult: The returned result from the optimization function
                covariance: The covariance matrix
                RiskFreeRate: The Risk free rate.
                
        Output:
                The output is a tuple which consists the following;
                
                OptimalWeights: A 1-Dimensional array representing the Optimal weights of the portfolios
                AnnualReturn: The Risk associated with the Portfolio
                AnnualRisk: The annual risk associated with the portfolio
                MaximalSharpeRatio: The optimized value of the Sharpe Ratio
                
    '''   
    
    
    # Optimal weights of each portfolio after optimization
    OptimalWeights = round(pd.DataFrame(MaximizedSharpeRatioResult.x, index=Tickers, columns=['Weights']).mul(1/sum(MaximizedSharpeRatioResult.x)), 4)

    # The risk of each portfolio which is the standard deviation
    Risk = OptimalWeights.transpose().dot(covariance).dot(OptimalWeights)

    # The expected monthly return
    ExpectedReturn = OptimalWeights.transpose().dot(Returns)

    
    # Annual risk associated with the Portfolio
    AnnualRisk =   (Risk).mul(12).apply(np.sqrt,axis=1)
    
    # Annual Return
    AnnualReturn = ExpectedReturn*12

    # Maximum Sharpe Ratio after Optimization
    
    MaximalSharpeRatio = round((ExpectedReturn.values[0][0] - RiskFreeRate)/Risk.apply(np.sqrt,axis=1).values[0][0], 4)
    
    return (OptimalWeights, AnnualReturn.values[0][0], AnnualRisk.values[0][0], MaximalSharpeRatio)

In [187]:
2**3

8

<br>
<br>

### Viewing Important results

In [188]:
# Size of portfolio
PortfolioSize = len(Returns)

# calling the optimization function with appropriate inputs
result = MaximizeSharpeRatio(Returns, covariance, RiskFreeRate, PortfolioSize)

# Reading the result of important data into variables

OptimalWeights, AnnualReturn, AnnualRisk, MaximalSharpeRatio = RiskAndSharpeRatio(result, covariance, RiskFreeRate)

<br>

### The Optimal Weights of Portfolio

In [189]:
OptimalWeights['PercentageWeight'] = OptimalWeights['Weights'].map('{:.2%}'.format)

OptimalWeights

,Weights,PercentageWeight
AAPL,0.0000,0.00%
ACN,0.0000,0.00%
ADBE,0.0183,1.83%
ADI,0.0000,0.00%
ADSK,0.0000,0.00%
AKAM,0.0000,0.00%
AMAT,0.0000,0.00%
AMD,0.0000,0.00%
ANSS,0.0000,0.00%
AVGO,0.1457,14.57%


<br>

### Annual Return

In [190]:
# Expressing the annual risk on a Dataframe both as a raw figure and percentage

AnnualReturn_df = pd.DataFrame([[AnnualReturn]], columns=['AnnualReturn(%)'])

AnnualReturn_df

,AnnualReturn(%)
0,3.8184


<br>

### Annual Risk

In [191]:
# Expressing the annual risk on a Dataframe both as a raw figure and percentage

AnnualRisk_df = pd.DataFrame([[AnnualRisk]], columns=['AnnualRisk(%)'])

AnnualRisk_df

,AnnualRisk(%)
0,0.682239


<br>

### Maximized Sharpe Ratio

In [192]:
 MaximalSharpeRatio

1.4227

<br>
<br>

The output shows that the maximal Sharpe Ratio attainable for the 52 portfolio is 1.42 which is good, going by practical standards. The annual expected portfolio return is 3.82% against an annualized risk of 0.68%.

To achieve this, the optimal capital allocations on the assets of  portfolio are as follows:


In [217]:
# Obtaining only those assets with non-zero values
InvestmentAllocation = OptimalWeights[['PercentageWeight']][OptimalWeights['Weights'] != 0]

# Adding the tickers title as the index title
InvestmentAllocation.index.name = 'Tickers'

# Renaming the column for comprehension
InvestmentAllocation.columns = ['PercentageAllocation']

InvestmentAllocation

,PercentageAllocation
Tickers,
ADBE,1.83%
AVGO,14.57%
CDNS,11.54%
FICO,16.07%
IT,0.97%
MPWR,12.09%
MSFT,7.71%
MSI,7.08%
NVDA,13.69%


<br>

**No investments need be made in the rest of the assets of  k -portfolio 1 since the optimal weights arrived at for these assets are 0.**

<br>
<br>


## References

1. Lesson 6:Sharpe Ratio based Portfolio Optimization    by DR G A VIJAYALAKSHMI PAI

https://www.kaggle.com/code/vijipai/lesson-6-sharpe-ratio-based-portfolio-optimization

2. https://www.youtube.com/watch?v=PgjkG6Xjz6Y

3. https://www.youtube.com/watch?v=tMNPYRhMqos

4. 